<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/huggingface/hf_nlp_07_main_NLP_tasks_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 主要的NLP任务(二)

### 翻译

这是另一个序列到序列的任务，这意味着它是一个可以表述为从一个序列到另一个序列的问题。从这个意义上说，这个问题非常接近总结，您可以将我们在这里看到的内容应用于其他序列到序列的问题，例如：

- 风格迁移：创建一个模型，将某种风格的文本翻译成另一种风格（例如，正式英语翻译成休闲英语，或者莎士比亚英语翻译成现代英语）

- 生成式问答：创建一个模型，根据给定的上下文生成问题的答案

在本节中，我们将在 KDE4 数据集（KDE 应用程序的本地化文件数据集）上微调预训练的 Marian 模型，以将其从英语翻译为法语（因为许多 Hugging Face 员工都说这两种语言）。

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

In [ ]:
!pip install transformers[torch] -U

#### 准备数据

我们将在本节中使用 KDE4 数据集，但只要您有要翻译的两种语言的句子对，您就可以轻松地调整代码以使用您自己的数据。

##### KDE4 数据集

In [ ]:
from datasets import load_dataset
# 下载数据集
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

# 查看数据集结构
raw_datasets

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [ ]:
# 使用训练数据的10%作为test
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets
# 将 "test" 重命名为 "validation"
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

[{'translation_text': 'Par défaut pour les threads élargis'}]

法国工程师在交谈时将大多数计算机科学专用单词保留为英语。例如，“threads”一词很可能出现在法语句子中，尤其是在技术对话中；但在这个数据集中，它已被翻译成更正确的“fils de Discussion”。还有“plugin”，它并不是正式的法语单词，但大多数母语人士都会理解它，而无需费心翻译。在 KDE4 数据集中，这个词已被翻译成法语，成为更官方的“module d’extension”

In [ ]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

然而，我们的预训练模型坚持使用紧凑且熟悉的英语单词：

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

##### 处理数据

我们数据的准备非常简单。只有一件事要记住；您需要确保分词器以输出语言（此处为法语）处理目标。您可以通过将目标传递给分词器的 __call__ 方法的 text_targets 参数来完成此操作。

###### 创建 tokenizer 对象

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [ ]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

###### 定义数据预处理函数

In [ ]:
max_length = 128

# 数据集的预处理函数
def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

> ⚠️ 我们不关注目标的注意力掩模，因为模型不会期望它。相反，与填充标记相对应的标签应设置为 -100 ，以便在损失计算中忽略它们。这将由我们的数据整理器稍后完成，因为我们正在应用动态填充

###### 数据预处理

In [ ]:
# 数据预处理
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

#### 使用 Trainer API 微调模型

我们在这里使用 Seq2SeqTrainer ，它是 Trainer 的子类将使我们能够使用 generate() 方法根据输入预测输出来正确处理评估。

##### 加载模型

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

##### 数据整理

我们需要一个数据整理器来处理动态批处理的填充。在这种情况下，我们不能像第 3 章那样只使用 DataCollatorWithPadding ，因为它只会填充输入（输入 ID、注意力掩码和令牌类型 ID）。我们的标签也应该填充到标签中遇到的最大长度。并且，用于填充标签的填充值应该是 -100 而不是标记器的填充标记，以确保在损失计算中忽略这些填充值。

我们将使用DataCollatorForSeq2Seq 完成。它采用 tokenizer 用于预处理输入，也采用 model。这是因为该数据整理器还将负责准备解码器输入 ID，这些 ID 是开头带有特殊标记的标签的移位版本。由于这种转变对于不同的体系结构略有不同，因此 DataCollatorForSeq2Seq 需要知道 model 对象：

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# 稍后会将data_collator 传递给 Seq2SeqTrainer

In [ ]:
# 验证上述说明
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch["labels"]

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

In [ ]:
batch["decoder_input_ids"]

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


##### 指标

用于翻译的传统指标是 BLEU 分数，BLEU 分数评估翻译与其标签的接近程度。它不衡量模型生成输出的可理解性或语法正确性，而是使用统计规则来确保生成输出中的所有单词也出现在目标中。还有一些规则，如果相同单词在目标中没有重复，则会对其重复进行惩罚（以避免模型输出类似 "the the the the the" 的句子），并输出比目标中的句子短的句子。

BLEU 的一个弱点是它期望文本已经被标记化，这使得比较使用不同标记器的模型之间的分数变得困难。因此，当今对翻译模型进行基准测试最常用的指标是 SacreBLEU，它通过标准化标记化步骤来解决这个弱点（以及其他弱点）。

###### 安装&加载指标

In [ ]:
!pip install sacrebleu

In [ ]:
import evaluate
# 加载评估指标
metric = evaluate.load("sacrebleu")
# 分数可以从 0 到 100，越高越好。

In [ ]:
# 测试
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

###### 使用指标计算

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

##### 微调模型

###### 构建微调参数

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


- f"marian-finetuned-kde4-en-to-fr" 模型输出目录
- evaluation_strategy="no": 指定评估策略。这里设置为"no"意味着在训练过程中不进行评估。
- save_strategy="epoch": 指定何时保存模型。这里设置为"epoch"，意味着在每个训练周期结束时保存模型。
- learning_rate=2e-5: 设置模型训练时的学习率。学习率是优化算法在参数空间中步进的大小。
- per_device_train_batch_size=32: 每个设备（GPU/CPU）在训练时的批量大小。批量大小是指每次模型更新前用于计算梯度的样本数量。
- per_device_eval_batch_size=64: 每个设备（GPU/CPU）在评估时的批量大小。这个参数在evaluation_strategy="no"时不会用到。
- weight_decay=0.01: 设置权重衰减（L2正则化）的强度，用于防止模型过拟合。
- save_total_limit=3:限制训练过程中保存的模型总数。这里设置为3，意味着最多只保存3个模型。
- num_train_epochs=3: 设置训练周期的数量。一个训练周期意味着整个训练数据集被完整地遍历一次。
- fp16=True ，是否使用混合精度（16位浮点数）训练。这可以加速训练并减少内存使用，同时保持几乎相同的模型精度。
- predict_with_generate=True 当设置为True时，使用模型的generate方法进行预测，这通常用于生成文本。
- push_to_hub=True 在每个 epoch 结束时将模型上传到 Hub

###### 构建训练器

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

###### 查看模型获得的分数

我们将首先查看模型获得的分数，以仔细检查我们的微调是否不会让事情变得更糟。

In [ ]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.6966999769210815,
 'eval_bleu': 39.264985846713074,
 'eval_runtime': 1480.1431,
 'eval_samples_per_second': 14.2,
 'eval_steps_per_second': 0.222}

###### 训练模型

In [ ]:
trainer.train()

Step,Training Loss
500,1.386400
1000,1.230700
1500,1.195500
2000,1.137700
2500,1.111000
3000,1.079100
3500,1.044200
4000,1.032000
4500,1.039100
5000,1.010000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=17736, training_loss=0.9368231966520248, metrics={'train_runtime': 4019.1469, 'train_samples_per_second': 141.19, 'train_steps_per_second': 4.413, 'total_flos': 1.1353213104095232e+16, 'train_loss': 0.9368231966520248, 'epoch': 3.0})

###### 再次查看模型得分

In [ ]:
trainer.evaluate(max_length=max_length)

##### 上传模型

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}


CommitInfo(commit_url='https://huggingface.co/leoli04/marian-finetuned-kde4-en-to-fr/commit/f727d3df8d659fe7c2a8db98a54ea1abc22d1b0b', commit_message='Training complete', commit_description='', oid='f727d3df8d659fe7c2a8db98a54ea1abc22d1b0b', pr_url=None, pr_revision=None, pr_num=None)

#### 自定义训练循环

##### 准备好训练用数据

###### 构建训练数据集和验证数据集
从数据集构建 DataLoader ，将数据集设置为 "torch" 格式

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

###### 重新实例化模型

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)


###### 构建优化器

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

###### 将上述对象传给accelerator.prepare

> 如果您想在 Colab 笔记本中的 TPU 上进行训练，则需要将所有这些代码移至训练函数中，并且不应执行任何实例化 Accelerator 的单元。

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

###### 计算训练步骤数

我们应该始终在准备数据加载器之后执行此操作，因为该方法将更改 DataLoader 的长度。

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

###### 将模型推送到仓库

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "marian-finetuned-kde4-en-to-fr-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
output_dir = "marian-finetuned-kde4-en-to-fr-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

##### 循环训练

为了简化其评估部分，我们定义了这个 postprocess() 函数，它接受预测和标签并将它们转换为我们的 metric 对象期望的字符串列表

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

#### 使用模型

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/marian-finetuned-kde4-en-to-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

### 文本摘要

在本节中，我们将了解如何使用 Transformer 模型将长文档压缩为摘要，这一任务称为文本摘要。这是最具挑战性的 NLP 任务之一，因为它需要一系列能力，例如理解长段落并生成捕获文档中主要主题的连贯文本。

我们将训练英语和西班牙语的双语模型。在本节结束时，您将拥有一个可以总结客户评论的模型。

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

In [ ]:
!pip install transformers[torch] -U

#### 准备多语言语料库

我们将使用多语言亚马逊评论语料库来创建双语摘要器。该语料库由六种语言的亚马逊产品评论组成，通常用于对多语言分类器进行基准测试。然而，由于每条评论都附有一个简短的标题，因此我们可以使用标题作为我们的模型学习的目标摘要！

##### 下载数据集

In [ ]:
from datasets import load_dataset

spanish_dataset = load_dataset("amazon_reviews_multi", "es")
english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset

In [ ]:
# 从训练集中抽取随机样本
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


show_samples(english_dataset)

在单个 GPU 上训练所有 400,000 条评论的摘要模型会花费太长时间，因此我们将专注于为单个产品领域生成摘要。为了了解我们可以选择哪些域，让我们将 english_dataset 转换为 pandas.DataFrame 并计算每个产品类别的评论数量：

In [ ]:
english_dataset.set_format("pandas")
english_df = english_dataset["train"][:]
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20]

英语数据集中最受欢迎的产品是家居用品、服装和无线电子产品。不过，为了坚持亚马逊的主题，让我们集中精力总结书评——毕竟，这就是该公司的成立基础！我们可以看到两个符合要求的产品类别（ book 和 digital_ebook_purchase ），因此让我们仅针对这些产品过滤两种语言的数据集。

##### 数据过滤

In [ ]:
def filter_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )

In [ ]:
english_dataset.reset_format()

In [ ]:
spanish_books = spanish_dataset.filter(filter_books)
english_books = english_dataset.filter(filter_books)
show_samples(english_books)

##### 将英语和西班牙语评论合并为单个 DatasetDict 对象

🤗 Datasets 提供了一个方便的 concatenate_datasets() 函数（顾名思义），它将两个 Dataset 对象堆叠在一起。

In [ ]:
from datasets import concatenate_datasets, DatasetDict

books_dataset = DatasetDict()

for split in english_books.keys():
  # 连接该分割的数据集
    books_dataset[split] = concatenate_datasets(
        [english_books[split], spanish_books[split]]
    )
    # 对结果进行洗牌，以确保我们的模型不会过度适合单一语言：
    books_dataset[split] = books_dataset[split].shuffle(seed=42)

# Peek at a few examples
show_samples(books_dataset)

##### 检查评论及其标题中的单词分布

下图显示了单词分布，我们可以看到标题严重偏向于 1-2 个单词：

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter7/review-lengths.svg"/>


为了解决这个问题，我们将过滤掉标题非常短的示例，以便我们的模型可以生成更有趣的摘要。由于我们正在处理英语和西班牙语文本，因此我们可以使用粗略的启发式方法在空格上分割标题，然后使用我们可靠的 Dataset.filter() 方法.

In [ ]:
books_dataset = books_dataset.filter(lambda x: len(x["review_title"].split()) > 2)


#### 文本摘要模型

如果你仔细想想，文本摘要与机器翻译是类似的任务：我们有一个像评论这样的文本正文，我们希望将其“翻译”成更短的版本，以捕获输入的显着特征。因此，大多数用于摘要的 Transformer 模型都采用我们在第 1 章中首次遇到的编码器-解码器架构，尽管也有一些例外

下表列出了一些流行的预训练模型，可以对其进行微调以进行汇总。

| Transformer model                                          | Description                                                  | Multilingual? |
| ---------------------------------------------------------- | ------------------------------------------------------------ | ------------- |
| [GPT-2](https://huggingface.co/gpt2-xl)                    | 尽管训练为自回归语言模型，但您可以通过在输入文本末尾附加“TL;DR”来使 GPT-2 生成摘要。 | ❌             |
| [PEGASUS](https://huggingface.co/google/pegasus-large)     | 使用预训练目标来预测多句子文本中的屏蔽句子。这个预训练目标比普通语言模型更接近总结，并且在流行的基准测试中得分很高。 | ❌             |
| [T5](https://huggingface.co/t5-base)                       | 通用 Transformer 架构，在文本到文本框架中制定所有任务；例如，模型总结文档的输入格式是 `summarize: ARTICLE` 。 | ❌             |
| [mT5](https://huggingface.co/google/mt5-base)              | T5的多语言版本，在多语言Common Crawl语料库（mC4）上进行预训练，涵盖101种语言。 | ✅             |
| [BART](https://huggingface.co/facebook/bart-base)          | 一种新颖的 Transformer 架构，具有编码器和解码器堆栈，经过训练可以重建损坏的输入，结合了 BERT 和 GPT-2 的预训练方案。 | ❌             |
| [mBART-50](https://huggingface.co/facebook/mbart-large-50) | BART 的多语言版本，经过 50 种语言的预训练。                  | ✅             |

正如您从该表中看到的，大多数用于摘要的 Transformer 模型（实际上是大多数 NLP 任务）都是单语言的。
有一类多语言 Transformer 模型（例如 mT5 和 mBART）可以解决这个问题。这些模型是使用语言建模进行预训练的，但有一点不同：它们不是在一种语言的语料库上进行训练，而是同时对 50 多种语言的文本进行联合训练！

我们将重点关注 mT5，这是一种基于 T5 的有趣架构，已在文本到文本框架中进行了预训练。在 T5 中，每个 NLP 任务都是根据提示前缀（如 summarize: ）来制定的，该前缀使模型适应生成的文本以适应提示。如下图所示，这使得 T5 非常通用，因为您可以用单个模型解决许多任务！


<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter7/t5.svg"/>

#### 预处理数据

我们的下一个任务是对我们的评论及其标题进行标记和编码。

###### 构建分词器

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# 测试 mT5 分词器

inputs = tokenizer("I loved reading the Hunger Games!")
inputs

In [ ]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

特殊的 Unicode 字符 ▁ 和序列结束标记 </s> 表明我们正在处理 SentencePiece 标记器，它基于第 6 章中讨论的 Unigram 分段算法。对于多语言语料库特别有用，因为它允许 SentencePiece 不受重音、标点符号以及许多语言（如日语）没有空白字符这一事实的影响。

In [ ]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["review_title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = books_dataset.map(preprocess_function, batched=True)

#### 文本摘要指标

最常用的指标之一是 ROUGE 分数（面向回忆的基础评估的缩写）。该指标背后的基本思想是将生成的摘要与通常由人类创建的一组参考摘要进行比较。

In [ ]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

##### 加载 ROUGE 指标

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
# 用 rouge_score.compute() 函数一次性计算所有指标
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

 rouge1 变体是一元组的重叠——这只是表示单词重叠的一种奇特方式，并且正是我们上面讨论的度量标准。
  rouge2 测量二元组之间的重叠（认为单词对的重叠），而 rougeL 和 rougeLsum 通过查找最长的单词匹配序列来测量单词的最长匹配序列生成的摘要和参考摘要中的公共子字符串。 rougeLsum 中的“总和”指的是该指标是在整个摘要上计算的，而 rougeL 是根据单个句子的平均值计算的。

In [ ]:
scores["rouge1"].mid

##### 建立强有力的基线

文本摘要的常见基线是简单地采用文章的前三个句子，通常称为 Lead-3 基线。我们可以使用句号来跟踪句子边界，但这对于像“U.S.”或“U.N.”这样的首字母缩略词来说会失败。- 因此我们将使用 nltk 库，其中包含更好的算法来处理这些情况。

In [ ]:
!pip install nltk

In [ ]:
import nltk

nltk.download("punkt")

接下来，我们从 nltk 导入句子标记器并创建一个简单的函数来提取评论中的前三个句子。文本摘要中的约定是用换行符分隔每个摘要，因此我们也将其包括在内并在训练示例中进行测试：

In [ ]:
from nltk.tokenize import sent_tokenize


def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])


print(three_sentence_summary(books_dataset["train"][1]["review_body"]))

In [ ]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]
    return metric.compute(predictions=summaries, references=dataset["review_title"])

In [ ]:
# 使用evaluate_baseline来计算验证集上的 ROUGE 分数
import pandas as pd

score = evaluate_baseline(books_dataset["validation"], rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict

#### 使用 Trainer API 微调 mT5

##### 加载模型

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

##### 定义训练参数

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

##### 为训练器提供 compute_metrics() 函数

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

##### 定义数据整理器

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    books_dataset["train"].column_names
)

features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

##### 训练模型

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

#### 使用 🤗 Accelerate 微调 mT5

##### 准备好训练的一切

In [ ]:
# 在数据集中将格式设置为 "torch"：由于 PyTorch 数据加载器需要批量张量
tokenized_datasets.set_format("torch")
# 再次实例化 DataCollatorForSeq2Seq
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# 实例化数据整理器并使用它来定义我们的数据加载器
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [ ]:
from torch.optim import AdamW
# 优化器
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator
# 将模型、优化器和数据加载器提供给 accelerator.prepare() 方法
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
# 后处理
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [ ]:
from huggingface_hub import get_full_repo_name

model_name = "test-bert-finetuned-squad-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
from huggingface_hub import Repository

output_dir = "results-mt5-finetuned-squad-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

##### 循环训练

文本摘要的训练循环与我们遇到的其他 🤗 Accelerate 示例非常相似，大致分为四个主要步骤：

- 通过迭代每个时期的 train_dataloader 中的所有示例来训练模型。
- 在每个时期结束时生成模型摘要，首先生成标记，然后将它们（和参考摘要）解码为文本。
- 使用我们之前看到的相同技术计算 ROUGE 分数。
- 保存检查点并将所有内容推送到集线器。

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

#### 使用您的微调模型

In [ ]:
from transformers import pipeline

hub_model_id = "huggingface-course/mt5-small-finetuned-amazon-en-es"
summarizer = pipeline("summarization", model=hub_model_id)

def print_summary(idx):
    review = books_dataset["test"][idx]["review_body"]
    title = books_dataset["test"][idx]["review_title"]
    summary = summarizer(books_dataset["test"][idx]["review_body"])[0]["summary_text"]
    print(f"'>>> Review: {review}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Summary: {summary}'")

In [ ]:
print_summary(100)

In [ ]:
print_summary(0)